<h1>LSTM Modeling</h1>

In [ ]:
# Process data and prepare dataframe. Change the file path to the location of the data file and the time step to the desired time step
from data_process import prepare_dataframe
file_path = "../../../CLT_data.csv"
time_step = "d"
df = prepare_dataframe(file_path, time_step)

<h2>Normalizing the data to a smaller range (0-1)</h2>
<p>After some research LSTM seem to work better with a smaller consistent range of values. In the case of calls, we get some days with lots and some with less</p>
<p>I apply some scalar methods from https://machinelearningmastery.com/how-to-scale-data-for-long-short-term-memory-networks-in-python/</p><br>
<h4>Things that normalizing data is good for:<h4>
<ul>
    <li>Speeds up learning</li>
    <li>Avoids exploding gradients (makes training unstable)</li>
    <li>Makes it easier to adjust the weights of each data point/node</li>
</ul>

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Normalize the call count data
scalar = MinMaxScaler(feature_range=(0, 1))
df["scaled_call_count"] = scalar.fit_transform(df["call_count"].values.reshape(-1, 1))

<h2>Create time-series generator</h2>
<p>Here we create generators to </p>
<p>https://machinelearningmastery.com/how-to-use-the-timeseriesgenerator-for-time-series-forecasting-in-keras/</p>

In [1]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
# The batch size here can be adjusted to optimize the model
batch_size_num = 20

# Create the initial generator that will be use
time_generator = TimeseriesGenerator(
    df["scaled_call_count"].values,
    df["scaled_call_count"].values,
    length=7,
    batch_size=batch_size_num    
    )

# Split the data into training and testing sets
split_index = int(len(df) * 0.8)  
training_data = df.iloc[:split_index]
testing_data = df.iloc[split_index:]

# Create the training and testing generators
training_generator = TimeseriesGenerator(
    training_data["scaled_call_count"].values,
    training_data["scaled_call_count"].values,
    length=7,
    batch_size=batch_size_num
    )

testing_generator = TimeseriesGenerator(
    training_data["scaled_call_count"].values,
    training_data["scaled_call_count"].values,
    length=7,
    batch_size=batch_size_num
    )



ModuleNotFoundError: No module named 'tensorflow'

<h2>Building the LSTM Model and Training</h2>
<p>Currently only uses 1 feature (time)</p>

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Build model (needs to be tuned more)
model = Sequential([
    LSTM(150, activation='relu', input_shape=(7, 1)),
    Dense(1),
    LSTM(150, activation='relu'),
    Dense(1)
])

# Train the model
model.fit(training_generator, epochs=10, validation_data=testing_generator)